<a href="https://colab.research.google.com/github/juancuevas-ops/analisis-molecular-covid19/blob/main/regrersion_logistica_molecular_jcuevas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Bio
from Bio import LogisticRegression
xs =    [[-53, -200.78],
          [117, -267.14],
          [57, -163.47],
          [16, -190.30],
          [11, -220.94],
          [85, -193.94],
          [16, -182.71],
          [15, -180.41],
          [-26, -181.73],
          [58, -259.87],
          [126, -414.53],
          [191, -249.57],
          [113, -265.28],
          [145, -312.99],
          [154, -213.83],
          [147, -380.85],
          [93, -291.13]]

In [ ]:
xs

[[-53, -200.78],
 [117, -267.14],
 [57, -163.47],
 [16, -190.3],
 [11, -220.94],
 [85, -193.94],
 [16, -182.71],
 [15, -180.41],
 [-26, -181.73],
 [58, -259.87],
 [126, -414.53],
 [191, -249.57],
 [113, -265.28],
 [145, -312.99],
 [154, -213.83],
 [147, -380.85],
 [93, -291.13]]

In [ ]:
 ys = [1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0]


In [ ]:
ys

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
modelo =  LogisticRegression.train(xs, ys)

Aquí, (xs ) y (ys) están los datos de entrenamiento: xs contiene las variables predictoras para cada par de genes y (ys) especifica si el par de genes pertenece al mismo operón ( 1, clase OP) oa diferentes operones ( 0, clase NOP). El modelo de regresión logística resultante se almacena en model, que contiene los pesos β 0 , β 1 y β 2 :

In [ ]:
modelo.beta

[8.98302901571447, -0.03596896044485089, 0.021813956629835193]

Tenga en cuenta que β 1 es negativo, ya que los pares de genes con una distancia entre genes más corta tienen una mayor probabilidad de pertenecer al mismo operón (clase OP). Por otro lado, β 2 es positivo, ya que los pares de genes que pertenecen al mismo operón suelen tener una puntuación de similitud más alta de sus perfiles de expresión génica. El parámetro β 0 es positivo debido a la mayor prevalencia de pares de genes de operón que de pares de genes sin operón en los datos de entrenamiento.

La función traintiene dos argumentos opcionales: update_fny typecode. Laupdate_fn puede usar para especificar una función de devolución de llamada, tomando como argumentos el número de iteración y la probabilidad de registro. Con la función de devolución de llamada, podemos, por ejemplo, realizar un seguimiento del progreso del cálculo del modelo (que utiliza una iteración de Newton-Raphson para maximizar la función de probabilidad logarítmica del modelo de regresión logística):

In [ ]:
def show_progress (iteración, loglikelihood):
        print ("Iteración:", iteración, "Función de probabilidad logarítmica:",loglikelihood)

In [ ]:
modelo = LogisticRegression.train (xs, ys, update_fn = show_progress)

Iteración: <built-in function iter> Función de probabilidad logarítmica: -11.78350206951907
Iteración: <built-in function iter> Función de probabilidad logarítmica: -7.158867676721056
Iteración: <built-in function iter> Función de probabilidad logarítmica: -5.76877209867943
Iteración: <built-in function iter> Función de probabilidad logarítmica: -5.113622943382592
Iteración: <built-in function iter> Función de probabilidad logarítmica: -4.748706424325651
Iteración: <built-in function iter> Función de probabilidad logarítmica: -4.5002607714604785
Iteración: <built-in function iter> Función de probabilidad logarítmica: -4.311277737371034
Iteración: <built-in function iter> Función de probabilidad logarítmica: -4.160150433955946
Iteración: <built-in function iter> Función de probabilidad logarítmica: -4.035617197847365
Iteración: <built-in function iter> Función de probabilidad logarítmica: -3.93073282192017
Iteración: <built-in function iter> Función de probabilidad logarítmica: -3.84087

El modelo de regresión logística clasifica yxcE , yxcD como pertenecientes al mismo operón (clase OP), mientras que se predice que yxiB , yxiA pertenecen a diferentes operones:

In [ ]:
print ("yxcE, yxcD:", LogisticRegression.classify (modelo, [6, -173.143442352])) 

yxcE, yxcD: 1


In [ ]:
print ("yxiB, yxiA:", LogisticRegression.classify (modelo, [ 309, -271.005880394])) 

yxiB, yxiA: 0


abla 16.2: Pares de genes adyacentes de estado de operón desconocido.
Par de genes	Distancia intergene x 1	Puntuación de expresión genética x 2
yxcE - yxcD	6	-173.143442352
yxiB - yxiA	309	-271.005880394

que, por cierto, concuerda con la literatura biológica).

Para averiguar qué tan seguros podemos estar en estas predicciones, podemos llamar a la calculatefunción para obtener las probabilidades (ecuaciones ( 16.2 ) y 16.3 ) para las clases OP y NOP. Para yxcE , yxcD encontramos

In [ ]:
q, p = LogisticRegression.calculate (modelo, [6, -173.143442352]) 
print ("clase OP: probabilidad =", p, "clase NOP: probabilidad =", q) 

clase OP: probabilidad = 0.9932421635025626 clase NOP: probabilidad = 0.006757836497437442


y para yxiB , yxiA

In [ ]:
q, p = LogisticRegression.calculate (modelo, [309, -271.005880394]) 
print ("clase OP: probabilidad =", p, "clase NOP: probabilidad =", q) 

clase OP: probabilidad = 0.00032121125181733257 clase NOP: probabilidad = 0.9996787887481827


Para tener una idea de la precisión de la predicción del modelo de regresión logística, podemos aplicarlo a los datos de entrenamiento:

In [ ]:
for i in range (len (ys)): 
        print ("True:", ys [i], "Predicted:", LogisticRegression.classify (modelo, xs [i])) 

True: 1 Predicted: 1
True: 1 Predicted: 0
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0


mostrando que la predicción es correcta para todos menos uno de los pares de genes. Se puede encontrar una estimación más confiable de la precisión de la predicción a partir de un análisis de omisión, en el que el modelo se vuelve a calcular a partir de los datos de entrenamiento después de eliminar el gen que se va a predecir:

In [ ]:
for i in range(len(ys)):
        model = LogisticRegression.train(xs[:i]+xs[i+1:], ys[:i]+ys[i+1:])
        print("True:", ys[i], "Predicted:", LogisticRegression.classify(modelo, xs[i]))


True: 1 Predicted: 1
True: 1 Predicted: 0
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 1 Predicted: 1
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0
True: 0 Predicted: 0


In [ ]:



from Bio import SeqIO


In [ ]:
record = SeqIO.read("/content/sequencebank.gb", "genbank")

In [ ]:
record

SeqRecord(seq=UnknownSeq(9609, character='N'), id='NC_005816.1', name='NC_005816', description='Yersinia pestis biovar Microtus str. 91001 plasmid pPCP1, complete sequence', dbxrefs=['BioProject:PRJNA224116', 'BioSample:SAMN02602970', 'Assembly:GCF_000007885.1'])

In [ ]:
!pip install reportlab
from reportlab.lib import colors
from reportlab.lib.units import cm
from Bio.Graphics import GenomeDiagram
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation

record = SeqIO.read("/content/sequencebank.gb", "genbank")

gd_diagram = GenomeDiagram.Diagram(record.id)
gd_track_for_features = gd_diagram.new_track(1, name="Annotated Features")
gd_feature_set = gd_track_for_features.new_set()

for feature in record.features:
    if feature.type != "gene":
        # Exclude this feature
        continue
    if len(gd_feature_set) % 2 == 0:
        color = colors.blue
    else:
        color = colors.lightblue
    gd_feature_set.add_feature(
        feature, sigil="ARROW", color=color, label=True, label_size=14, label_angle=0
    )

# I want to include some strandless features, so for an example
# will use EcoRI recognition sites etc.
for site, name, color in [
    ("GAATTC", "EcoRI", colors.green),
    ("CCCGGG", "SmaI", colors.orange),
    ("AAGCTT", "HindIII", colors.red),
    ("GGATCC", "BamHI", colors.purple),
]:
    index = 0
    while True:
        index = record.seq.find(site, start=index)
        if index == -1:
            break
        feature = SeqFeature(FeatureLocation(index, index + len(site)))
        gd_feature_set.add_feature(
            feature,
            color=color,
            name=name,
            label=True,
            label_size=10,
            label_color=color,
        )
        index += len(site)

gd_diagram.draw(format="linear", pagesize="A4", fragments=4, start=0, end=len(record))
gd_diagram.write("plasmid_linear_nice.pdf", "PDF")
gd_diagram.write("plasmid_linear_nice.eps", "EPS")
gd_diagram.write("plasmid_linear_nice.svg", "SVG")

gd_diagram.draw(
    format="circular",
    circular=True,
    pagesize=(20 * cm, 20 * cm),
    start=0,
    end=len(record),
    circle_core=0.5,
)
gd_diagram.write("plasmid_circular_nice.pdf", "PDF")
gd_diagram.write("plasmid_circular_nice.eps", "EPS")
gd_diagram.write("plasmid_circular_nice.svg", "SVG")
#And the output:

In [ ]:
print(record)

ID: NC_005816.1
Name: NC_005816
Description: Yersinia pestis biovar Microtus str. 91001 plasmid pPCP1, complete sequence
Database cross-references: BioProject:PRJNA224116, BioSample:SAMN02602970, Assembly:GCF_000007885.1
Number of features: 19
/molecule_type=DNA
/topology=circular
/data_file_division=CON
/date=25-OCT-2020
/accessions=['NC_005816']
/sequence_version=1
/keywords=['RefSeq']
/source=Yersinia pestis biovar Microtus str. 91001
/organism=Yersinia pestis biovar Microtus str. 91001
/taxonomy=['Bacteria', 'Proteobacteria', 'Gammaproteobacteria', 'Enterobacterales', 'Yersiniaceae', 'Yersinia']
/references=[Reference(title='Genetics of metabolic variations between Yersinia pestis biovars and the proposal of a new biovar, microtus', ...), Reference(title='Complete genome sequence of Yersinia pestis strain 91001, an isolate avirulent to humans', ...), Reference(title='Direct Submission', ...)]
/comment=REFSEQ INFORMATION: The reference sequence is identical to
AE017046.1.
The annota